<a href="https://colab.research.google.com/github/sohyun111/fine_dust/blob/main/%EB%AF%B8%EC%84%B8%EB%A8%BC%EC%A7%80%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
f = open('/content/drive/MyDrive/인공지능과제/서울미세먼지.csv','r')

In [ ]:
!ls -al
!pwd

In [ ]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt


dust_data = pd.read_csv(f,header=0)
seq = dust_data[['SEOUL']].to_numpy()
data_length = len(seq)
print('data length:', data_length)
print('The first 20 data:\n', seq[:20])

plt.figure(figsize=(20,10))
plt.plot(seq[:,0],color='red')
plt.title('Fine Dust in SEOUL (2010.01 - 2023.02)')
plt.xlabel('Days')
plt.ylabel('Fine Dust')
plt.show()

In [ ]:
import numpy as np

def seq2dataset(seq, window, horizon):
  X=[]
  Y=[]
  for i in range(len(seq)-(window + horizon) + 1):
    x = seq[i:(i + window)]
    y = (seq[i + window + horizon - 1])
    X.append(x); Y.append(y)

  return np.array(X), np.array(Y)
w = 1
h = 1

X, Y = seq2dataset(seq,w,h)
print(X.shape, Y.shape)
print(X[0], Y[0])

In [ ]:
split_train = int(len(X)*0.7)
split_valid = int(len(X)*0.9)

X_train = X[:split_train, :, [0]]; y_train=Y[:split_train, [0]]

X_valid = X[split_train:split_valid,:, [0]]; y_valid=Y[split_train:split_valid,[0]]

X_test = X[split_valid:,:, [0]]; y_test=Y[split_valid:,[0]]

print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)
print(X_test.shape, y_test.shape)

In [ ]:
import tensorflow as tf
from tensorflow import keras

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[w, 1]),
    keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, y_train, epochs=200,
                    validation_data=(X_valid, y_valid))

In [ ]:
import math
eval = model.evaluate(X_test, y_test)

math.sqrt(eval)

In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

def plot_learning_curves(loss, val_loss):
    plt.plot(np.arange(len(loss)) + 0.5, loss, "b.-", label="Training loss")
    plt.plot(np.arange(len(val_loss)) + 1, val_loss, "r.-", label="Validation loss")
    plt.gca().xaxis.set_major_locator(mpl.ticker.MaxNLocator(integer=True))
    plt.axis([1, 200, 100, 3000])
    plt.legend(fontsize=14)
    plt.xlabel("Epochs")
    plt.ylabel("Loss (MSE)")
    plt.grid(True)

def plot_series(series, y=None, y_pred=None, x_label="$t$", y_label="$x(t)$"):
    plt.plot(series, ".-")
    if y is not None:
        plt.plot(n_steps, y, "bx", markersize=10)
    if y_pred is not None:
        plt.plot(n_steps, y_pred, "ro")
    plt.grid(True)
    if x_label:
        plt.xlabel(x_label, fontsize=16)
    if y_label:
        plt.ylabel(y_label, fontsize=16, rotation=0)

In [ ]:
plot_learning_curves(history.history["loss"], history.history["val_loss"])
plt.show()

In [ ]:
n_steps = 7

print(X_valid.shape)
y_pred = model.predict(X_valid)
print(y_pred.shape)

print("gt:", y_valid[0])
print("pred:", y_pred[0])

plot_series(X_valid[0, :, 0], y_valid[0, 0], y_pred[0, 0])
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow import keras

model = keras.models.Sequential([
    keras.layers.SimpleRNN(1, activation='relu', input_shape=[None, 1]),
    keras.layers.Dense(1)
])

model.summary()

optimizer = keras.optimizers.Adam(learning_rate=0.003)
model.compile(loss="mse", optimizer=optimizer)
history = model.fit(X_train, y_train, epochs=200,
                    validation_data=(X_valid, y_valid))

In [ ]:
import math
eval = model.evaluate(X_test, y_test)

math.sqrt(eval)

In [ ]:
plot_learning_curves(history.history["loss"], history.history["val_loss"])
plt.show()

In [ ]:
n_steps = 7

print(X_valid.shape)
y_pred = model.predict(X_valid)
print(y_pred.shape)

print("gt:", y_valid[0])
print("pred:", y_pred[0])

plot_series(X_valid[0, :, 0], y_valid[0, 0], y_pred[0, 0])
plt.show()